## Guide to performing <img src="images/logo.png" alt="Alt text" style="display: inline; width: 300px; vertical-align: middle; margin-top: -10px;"/>

In this notebook, we will use the example of CO on MgO to illustrate how autoSKZCAM can be used to get the adsorption enthalpy accurately and efficiently. It requires only the execution of a few simple functions, all of which are found under ``autoSKZCAM.recipes``. We will go over the set of functions to execute in a chronological order, starting with using the SKZCAM protocol to obtain the interaction energy, before going over the DFT ensemble.

First, let's import the set of functions, they will be run as follows:
``skzcam_initialize`` &rarr; ``skzcam_eint_flow`` &rarr; ``skzcam_analyse`` &rarr; ``dft_ensemble_flow`` &rarr; ``dft_analyse`` &rarr; ``get_final_autoSKZCAM_Hads``. 

In [1]:
from autoSKZCAM.recipes import dft_ensemble_analyse, dft_ensemble_flow, get_final_autoSKZCAM_Hads, skzcam_analyse, skzcam_eint_flow, skzcam_initialize

## 1. Initialize the embedded cluster with: `skzcam_initialize`

In [2]:
EmbeddedCluster = skzcam_initialize(adsorbate_indices =[0,1],slab_center_indices=[32],atom_oxi_states = {'Mg':2, 'O':-2},adsorbate_slab_file='POSCAR_CO_on_MgO', pun_filepath='ChemShell_EmbeddedCluster.pun')

Initializes the SKZCAM protocol to generate the embedded clusters. ChemShell will be executed if it hasn't already been run to produce a .pun file containing the embedded cluster positions. It is required that you have installed py-ChemShell v20.0.0 and up at: https://chemshell.org/licence/. The main output of `skzcam_initialize`` is the EmbeddedCluster object, which contains an ASE Atoms object of the electrostatic embedded cluster generated by ChemShell. See https://www.chemshell.org/static_files/py-chemshell/manual/build/html/cluster.html for more information and an explanation of ChemShell specific inputs.

The code executed above assumes ChemShell has been run prior to produce a .pun file called `ChemShell_EmbeddedCluster.pun`. Use the code below if ChemShell has not been executed already for this system:

```python
EmbeddedCluster = skzcam_initialize(adsorbate_indices =[0,1],slab_center_indices=[32],atom_oxi_states = {'Mg':2, 'O':-2},adsorbate_slab_file='POSCAR_CO_on_MgO', run_chemshell=True)
```





<div style="float: right; margin: 0 0 10px 10px;">
  <img src="images/co_on_mgo.png" alt="Alt text" width="200"/>
</div>

**Parameters**  
- **adsorbate_indices** (`list[int]`, **required**):  
  Indices of the atoms that make up the adsorbate molecule.

- **slab_center_indices** (`list[int]`, **required**):  
  Indices of the atoms that make up the “center” of the slab right beneath the adsorbate.

- **atom_oxi_states** (`dict[ElementStr, int]`, **required**):  
  Dictionary with element symbols as keys and their oxidation states as values.

- **adsorbate_slab_file** (`str | Path`, **required**):  
  Path to the file containing the adsorbate molecule on the surface slab (any format readable by ASE).

- **pun_filepath** (`str | Path`, default: `./ChemShell_EmbeddedCluster.pun`):  
  Path to the `.pun` file containing coordinates/charges if already generated by ChemShell. If `None`, ChemShell will be called to create this file.

- **run_chemshell** (`bool`, default: `False`):  
  If `True`, runs ChemShell to create or update the `.pun` file if it doesn’t already exist.

- **chemsh_radius_active** (`float`, default: `40.0`):  
  Radius of the active region (in Å) for charge fitting to ensure correct Madelung potential. Can be relatively large.

- **chemsh_radius_cluster** (`float`, default: `60.0`):  
  Total radius (in Å) of the embedded cluster.

- **chemsh_bq_layer** (`float`, default: `6.0`):  
  Height (in Å) above the surface at which additional fitting point charges are placed to improve the electrostatic potential near the adsorbate.

- **write_xyz_file** (`bool`, default: `False`):  
  If `True`, writes out the XYZ file for the final cluster.

**Returns**
<code>CreateEmbeddedCluster</code>

An object representing the constructed embedded cluster. See [here](https://github.com/benshi97/autoSKZCAM/blob/main/src/autoSKZCAM/embed.py) for more details. 
<details>
  <summary>Click for object attributes</summary>

- <code>adsorbate_indices</code>: Indices of atoms in the adsorbate.
- <code>slab_center_indices</code>: Indices of atoms at the center of the slab.
- <code>slab_indices</code>: Indices of atoms in the slab (initialized as <code>None</code>).
- <code>atom_oxi_states</code>: Oxidation states of atoms.
- <code>adsorbate_slab_file</code>: File containing the geometry of the adsorbate-slab complex.
- <code>pun_filepath</code>: Filepath to the <code>.pun</code> file for ChemShell.
- <code>skzcam_calcs</code>: Dictionary for calculator information for SKZCAM clusters (<code>None</code> by default).
- <code>OniomInfo</code>: Dictionary for information on ONIOM layers (<code>None</code> by default).
- <code>adsorbate</code>: <code>Atoms</code> object representing the adsorbate (<code>None</code> by default).
- <code>slab</code>: <code>Atoms</code> object representing the slab (<code>None</code> by default).
- <code>adsorbate_slab</code>: <code>Atoms</code> object for the adsorbate-slab complex (<code>None</code> by default).
- <code>adsorbate_slab_embedded_cluster</code>: <code>Atoms</code> object for the embedded adsorbate-slab cluster (<code>None</code> by default).
- <code>slab_embedded_cluster</code>: <code>Atoms</code> object for the embedded slab cluster (<code>None</code> by default).
- <code>quantum_cluster_indices_set</code>: List of quantum cluster indices (<code>None</code> by default).
- <code>ecp_region_indices_set</code>: List of indices for the ECP region (<code>None</code> by default).
- <code>adsorbate_vector_from_slab</code>: Vector representing the position of the adsorbate relative to the slab (calculated during initialization).
- <code>center_position</code>: Center position of the cluster (calculated during initialization).

</details>

## 2. Perform the SKZCAM protocol with: `skzcam_eint_flow`

In [ ]:
OniomInfo = {
        "Bulk MP2": {
            "ll": None,
            "hl": {
                "method": "MP2",
                "frozen_core": "valence",
                "basis": "CBS(DZ//TZ)",
                "max_cluster_num": 5,
                "code": "orca",
            },
        },
        "Delta_Basis and Delta_Core": {
            "ll": {
                "method": "MP2",
                "frozen_core": "valence",
                "basis": "CBS(DZ//TZ)",
                "max_cluster_num": 3,
                "code": "orca",
            },
            "hl": {
                "method": "MP2",
                "frozen_core": "semicore",
                "basis": "CBS(TZ//QZ)",
                "max_cluster_num": 3,
                "code": "orca",
            },
        },
        "FSE Error": {
            "ll": {
                "method": "MP2",
                "frozen_core": "valence",
                "basis": "DZ",
                "max_cluster_num": 5,
                "code": "orca",
            },
            "hl": {
                "method": "MP2",
                "frozen_core": "valence",
                "basis": "DZ",
                "max_cluster_num": 7,
                "code": "orca",
            },
        },
        "DeltaCC": {
            "ll": {
                "method": "LMP2",
                "frozen_core": "valence",
                "basis": "CBS(DZ//TZ)",
                "max_cluster_num": 3,
                "code": "mrcc",
            },
            "hl": {
                "method": "LNO-CCSD(T)",
                "frozen_core": "valence",
                "basis": "CBS(DZ//TZ)",
                "max_cluster_num": 3,
                "code": "mrcc",
            },
        },
    }

skzcam_eint_flow(EmbeddedCluster,OniomInfo,dryrun=True, )